# target : tip

In [1]:
import pymysql.cursors
import pandas as pd

In [2]:
# 불러오기
conn = pymysql.connect(host='localhost', user='zeros', 
                       password='160525', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:
    with conn.cursor() as curs:
        sql = "select * from tips;"
        curs.execute(sql)
        rs = curs.fetchall()
        df = pd.DataFrame(rs)
finally:
    conn.close()

# encoding

In [3]:
df["sex"] = df["sex"].replace({"Female" : 0, "Male" : 1})
df["smoker"] = df["smoker"].replace({"No" : 0, "Yes" : 1})
df["day"] = df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3})
df["time"] = df["time"].replace({"Lunch" : 0, "Dinner" : 1})
df['tip_rate']= df['tip']/df['total_bill']

In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           1
smoker        0
day           1
time          2
size          0
tip_rate      0
dtype: int64

# null값 처리

In [5]:
from numpy import isnan
from sklearn.impute import SimpleImputer

In [6]:
# y=타겟
y = df['tip']
X = df.drop('tip',axis=1) #학습할 x에서 tip을 드랍

In [7]:
data = X.values
y = y.values

In [8]:
#NULL값 확인
sum(isnan(data).flatten()), sum(isnan(y).flatten())

(4, 0)

In [9]:
#IMPUTED DATA SET(널없는 데이터)
imputer = SimpleImputer(strategy='median')
imputer.fit(data)
data_trans = imputer.transform(data) 

In [10]:
df1 = pd.DataFrame(data_trans)

In [11]:
df1.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64

In [12]:
df1

,0,1,2,3,4,5,6
0,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
1,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
2,21.01,1.0,0.0,3.0,1.0,3.0,0.166587
3,23.68,1.0,0.0,3.0,1.0,2.0,0.139780
4,24.59,0.0,0.0,3.0,1.0,4.0,0.146808
...,...,...,...,...,...,...,...
244,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
245,16.99,1.0,0.0,2.0,1.0,3.0,0.059447
246,16.99,0.0,0.0,0.0,1.0,3.0,0.059447
247,16.99,1.0,0.0,2.0,1.0,3.0,0.059447


In [13]:
# null 값 처리 후 원상복구
df1 = pd.DataFrame(data_trans, columns=['total_bill','sex','smoker','day','time','size','tip_rate'])
df1

,total_bill,sex,smoker,day,time,size,tip_rate
0,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
1,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
2,21.01,1.0,0.0,3.0,1.0,3.0,0.166587
3,23.68,1.0,0.0,3.0,1.0,2.0,0.139780
4,24.59,0.0,0.0,3.0,1.0,4.0,0.146808
...,...,...,...,...,...,...,...
244,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
245,16.99,1.0,0.0,2.0,1.0,3.0,0.059447
246,16.99,0.0,0.0,0.0,1.0,3.0,0.059447
247,16.99,1.0,0.0,2.0,1.0,3.0,0.059447


# RFE
- 중요도가 낮은 변수를 제거하는 방식.

In [14]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [15]:
rfe = RFE(estimator=SVR(kernel="linear"), n_features_to_select=4)

In [16]:
# fit RFE
rfe.fit(df1, y)

RFE(estimator=SVR(kernel='linear'), n_features_to_select=4)

In [17]:
df1.shape

(249, 7)

In [18]:
for i in range(df1.shape[1]):
  print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected=True, Rank: 1
Column: 1, Selected=False, Rank: 2
Column: 2, Selected=False, Rank: 3
Column: 3, Selected=True, Rank: 1
Column: 4, Selected=True, Rank: 1
Column: 5, Selected=False, Rank: 4
Column: 6, Selected=True, Rank: 1


In [19]:
df1.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_rate'], dtype='object')

In [20]:
# 랭킹에 따라서 랭킹1위를 제외한 나머지는 숙청
df1.drop(['sex','smoker','size'], axis=1)

,total_bill,day,time,tip_rate
0,16.99,3.0,1.0,0.059447
1,10.34,3.0,1.0,0.160542
2,21.01,3.0,1.0,0.166587
3,23.68,3.0,1.0,0.139780
4,24.59,3.0,1.0,0.146808
...,...,...,...,...
244,16.99,3.0,1.0,0.059447
245,16.99,2.0,1.0,0.059447
246,16.99,0.0,1.0,0.059447
247,16.99,2.0,1.0,0.059447


# PCA = 공분산값을 구해서 아이겐벨류값을 얻어서 그걸 비교해서 위에서 부터 큰값을 리턴


In [24]:
from sklearn.decomposition import PCA

In [25]:
data_trans.shape

(249, 7)

In [26]:
# define the transform
trans = PCA(n_components=7)

# transform the data
X_dim = trans.fit_transform(data_trans)


In [32]:
# 다시 데이터프레임으로 봐버리기
df2 = pd.DataFrame(pd.DataFrame(X_dim, columns=['total_bill','sex','smoker','day','time','size','tip_rate']))
df2

,total_bill,sex,smoker,day,time,size,tip_rate
0,-2.744721,-1.293859,-0.385544,-0.716872,0.472551,-0.150171,-0.113937
1,-9.307505,-1.621448,0.986024,0.035812,-0.299803,-0.090620,-0.027552
2,1.337007,-1.319279,0.346178,-0.162222,-0.335340,-0.122639,0.005353
3,3.936999,-1.198487,-0.789368,-0.393075,-0.438280,-0.160389,-0.011742
4,4.964327,-1.169089,1.109623,-0.495124,0.635585,-0.114444,-0.013445
...,...,...,...,...,...,...,...
244,-2.744721,-1.293859,-0.385544,-0.716872,0.472551,-0.150171,-0.113937
245,-2.696319,-0.493042,0.621057,-0.046833,-0.396710,0.221620,-0.096314
246,-2.750519,1.480134,0.691834,-0.410993,0.345047,0.865933,-0.070356
247,-2.696319,-0.493042,0.621057,-0.046833,-0.396710,0.221620,-0.096314


StandardScaler

In [369]:
df2.corr()

,0,1,2,3
0,1.000000e+00,2.076775e-17,2.067356e-18,1.860237e-17
1,2.076775e-17,1.000000e+00,-4.346025e-16,-3.005123e-16
2,2.067356e-18,-4.346025e-16,1.000000e+00,-2.635664e-16
3,1.860237e-17,-3.005123e-16,-2.635664e-16,1.000000e+00


# Regression Feature Selection

In [294]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [295]:
fs = SelectKBest(score_func=f_regression, k=4)
# apply feature selection
X_selected = fs.fit_transform(df1, y)

print(X_selected.shape)

(249, 4)


In [296]:
df3 = pd.DataFrame(X_selected)

In [297]:
df3

,0,1,2,3
0,16.99,3.0,2.0,0.059447
1,10.34,3.0,3.0,0.160542
2,21.01,3.0,3.0,0.166587
3,23.68,3.0,2.0,0.139780
4,24.59,3.0,4.0,0.146808
...,...,...,...,...
244,16.99,3.0,2.0,0.059447
245,16.99,2.0,3.0,0.059447
246,16.99,0.0,3.0,0.059447
247,16.99,2.0,3.0,0.059447
